In [10]:
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpl
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
'''
戦略
ボラティリティー低下: 標準偏差が一定以下、移動平均乖離率、高値と安値の幅
でシグナルを出す
上放れした時点で買い
ストップは順次あげていく
'''

#install symbols nasdaq100の銘柄コード
df = pd.read_csv('../symbols/nasdaq100.csv')

sign = []
for symbol in df['symbol']:
    sign.append(symbol)
    

# sign = ['6976.T','6036.T','6361.T','ATVI', 'AMD', 'ALXN', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AMGN', 'ADI'
#         , 'ANSYS', 'ANSS', 'AAPL', 'AMAT', 'TEAM', 'ADSK', 'ADP', 'BIDU', 'BIIB', 'BKNG'
#         , 'AVGO','CDNS', 'CDW', 'CDW', 'CERN', 'CHTR', 'CHKP', 'CTAS', 'CSCO', 'CTSH', 'CMCSA'
#         , 'CPRT', 'COST', 'CSX', 'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 'FB', 'FAST'
#         , 'FISV', 'FOXA', 'FOX', 'GILD', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD'
#         , 'KDP', 'KLAC', 'KHC', 'LRCX', 'LULU', 'MAR', 'MRVL', 'MTCH', 'MXIM', 'MELI', 'MCHP', 'MU'
#         , 'MSFT', 'MRNA', 'MDLZ', 'MNST', 'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'OKTA', 'PCAR', 'PAYX'
#         , 'PYPL', 'PTON', 'PEP', 'PDD', 'QUALCOMM', 'QCOM', 'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK'
#         , 'SBUX', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'TCOM', 'VRSN', 'VRSK', 'VRTX', 'WBA', 'WDAY', 'XEL', 'XLNX'
#         , 'ZM']

# sign = ['4477.T','6976.T','6036.T']
    

totalresult = []

for i in sign:
    stock_code = i 
    start = '2015-01-01'
    end = '2021-02-02'
    df = yf.download(stock_code, start, end, interval='1d') 
    if df.empty:
        print('{} is empty'.format(stock_code))
        results = []
        resultNum = 7
        for num in range(resultNum):
            results.append(np.nan)
        totalresult.append(results)
        continue;
        
    
    
    print(i)
    term=81
    # chek the highest price in the past {term} times
    df['Highest'+str(term)] = df.iloc[:, 4].rolling(window=term).max()
#     print(df.head())
    
        
    
   
    
    # caluculation ATR
    h, l, c_prev = df['High'], df['Low'], pd.Series(df['Close']).shift(1)
    tr = np.max([h - l, (c_prev - h).abs(), (c_prev - l).abs()], axis=0)
    atr = pd.Series(tr).rolling(14).mean().bfill().values
    
    # caluculation range
    df['mean'+str(14)] = df.iloc[:, 4].rolling(window=14).mean()
    df['Highest'+str(14)] = df.iloc[:, 4].rolling(window=14).max()
    df['Lowest'+str(14)] = df.iloc[:, 4].rolling(window=14).min()
    df['range'+str(14)] = 100 * (df['Highest'+str(14)] - df['Lowest'+str(14)] )/ df['mean'+str(14)]
    
    
    
    
     # 損切り率
    stopLoss = 0.94
    
    # signal 判断
    signal_arart = 0
 
    
    #買い日と売り日
    buy_days = []
    sell_days = []
    signal_days = []
    
    #holding period list
    holding_period_list = []


    #calucurate Bband
    df['MA'] = df['Adj Close'].rolling(window=25).mean()
    df['STD'] = df['Adj Close'].rolling(window=25).std()
    df['Upper'] = df['MA'] + (df['STD'] * 2)
    df['Lower'] = df['MA'] - (df['STD'] * 2)
    
    
     # calculation SMA
    short_sma = 25
    long_sma = 50
    verylong_sma = 250
    #'term'日間の移動平均線
    df['SMA'+str(short_sma)] = df['Adj Close'].rolling(window=short_sma).mean()
    df['SMA'+str(long_sma)] = df['Adj Close'].rolling(window=long_sma).mean()
    df['SMA'+str(verylong_sma)] = df['Adj Close'].rolling(window=verylong_sma).mean()
    
    df['slope_sma'] = np.log( df['SMA'+str(short_sma)] / df['SMA'+str(short_sma)].shift(1) ) 
    
    
    #daviation_rate
    df['daviation_rate'] = 100 * (df['Adj Close'] - df['SMA'+str(short_sma)]) / df['SMA'+str(short_sma)]

    #Standard deviation normalization
    df['Standard_deviation_normalization'] = 100 * 2 * df['STD'] / df['SMA'+str(short_sma)]
   
    buy_position = False
    percentChange = []
    signal_alart = 0
    for i in range(1, len(df)):
        close = df['Adj Close'][i]
        STD = df['STD'][i]
        high_price = df['High'][i]
        low_price = df['Low'][i]
        open_price = df['Open'][i]
        sma = df['SMA'+str(short_sma)][i]
        SDN = df['Standard_deviation_normalization'][i]
        daviation_rate = df['daviation_rate'][i]
        Highest = df['Highest'+str(term)][i-1]
        slope_sma = df['slope_sma'][i]
        true_range = df['range'+str(14)][i]
        

    
        if np.isnan(df['High'][i]) == False:
            if -3 < daviation_rate < 3 and true_range < 6:
#                 print('low volatility')
# and close > 0.95 * Highest SDN < 10 and 
                signalday = pd.to_datetime(df.index[i])
                signal_days.append(signalday)
#                 print(signalday)
                signal_point = i
                signal_alart =1
        
        if signal_alart == 1 and slope_sma > 0 and i - signal_point < 7 and open_price > Highest\
            and buy_position == False and open_price < close :
            buy_position = True
            buyday = pd.to_datetime(df.index[i])
            buy_days.append(buyday)
            buy_price = close
            buy_stopLine = close * stopLoss
            stop_point = i
            current_stop_level = 0
            print('buy in {} at {}'.format(buyday,buy_price))
            signal_arart = 0
           
        
        #adjustment stopline
        if buy_position == True and close > buy_price * 1.05 and current_stop_level == 0:
            buy_stopLine = buy_price
            current_stop_level = 1
            stop_point = i
            print('up stopline1 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.1 and current_stop_level == 1:
            buy_stopLine = buy_price * 1.05
            current_stop_level = 2
            stop_point = i
            print('up stopline2 in {} at {}'.format(df.index[i],buy_stopLine))
           
            
        if buy_position == True and close > buy_price * 1.2 and current_stop_level == 2:
            buy_stopLine = buy_price * 1.1
            current_stop_level = 3
            stop_point = i
            print('up stopline3 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.3 and current_stop_level == 3:
            buy_stopLine = close * 0.85
            current_stop_level = 4
            stop_point = i
            print('up stopline4 in {} at {}'.format(df.index[i],buy_stopLine))
            
        if buy_position == True and close > buy_price * 1.5 and current_stop_level == 4:
            buy_stopLine = close * 0.8
            current_stop_level = 5  
            stop_point = i
            print('up stopline5 in {} at {}'.format(df.index[i],buy_stopLine))
            
            
        if buy_position == True and close > Highest and current_stop_level == 5:
            buy_stopLine = close * 0.8
            current_stop_level = 5  
            stop_point = i
            print('up stopline6 in {} at {}'.format(df.index[i],buy_stopLine))
            
            
            
         # 損切り or Profit taking
        if buy_position == True and open_price < buy_stopLine and i != stop_point :
            buy_position = False 
            percent = (open_price/buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            print('sell in {} at {} | profit_one is {}'.format(sellday,open_price,percent))
            
        elif buy_position == True and close < buy_stopLine and i != stop_point:
            buy_position = False 
            percent = (buy_stopLine/buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            holding_period_list.append(holding_period.days)
            print('sell in {} at {} | profit_two is {}'.format(sellday,buy_stopLine,percent))

        #  最終日にまだ持ってたら利確する
        if i == (len(df)-1) and buy_position == True:
            buy_position = False
            percent = (close / buy_price - 1) * 100
            percentChange.append(percent)
            sellday = pd.to_datetime(df.index[i])
            sell_days.append(sellday)
            holding_period = sellday - buyday
            holding_period_list.append(holding_period.days)
            print('Date:'+str(i),'Last Day Buy at the price {}'.format(close))

       
    # statistic
#     ------------------------------------------
    gains = 0
    losses = 0
    numlosses = 0
    numgains = 0
    total_return = 1


    for i in percentChange:
        if i > 0:
            numgains += 1
            gains += i
        else:
            numlosses += 1
            losses += i
        total_return = total_return * ((i / 100) + 1)
        
        
    total_return = round((total_return - 1)*100, 2)
    if numgains > 0:
        ave_gain = gains / numgains
#         max_return = max(percentChange)
    else:
        ave_gain = 0
        max_return = 'unknown'
    
    if numlosses > 0:
        ave_loss = losses / numlosses
#         max_loss = min(percentChange)
        risk_reward_retio = - ave_gain / ave_loss
    else:
        ave_loss = 0
        max_loss = 'unknown'
        risk_reward_retio = 'inf'
    
    if numgains > 0 or numlosses > 0:
        batting_retio = numgains / (numgains + numlosses)
    else:
        batting_retio = 0
    
    trades = numgains + numlosses
    ave_total = ave_gain * batting_retio + ave_loss * (1 - batting_retio)
    
    #average holding period
    if len(holding_period_list) != 0:   
        average_holding_period = sum(holding_period_list)/len(holding_period_list)
        
    else :
        average_holding_period = np.nan
    

    results = [numgains+numlosses, round(total_return,2), round(ave_gain,2), round(ave_loss,2)
               ,round(batting_retio,2) ,round(ave_total,2) ,round(average_holding_period,2)]
    
    if not results is None:
         totalresult.append(results)
            
#     columns = ['trades', 'Total return', 'Average Gain', 'Average Loss', '勝率' ,'Average gain and profit','average holding period']
#     df_result = pd.DataFrame(totalresult, columns=columns, index = sign)
#     print(df_result)
            


    #plot figure --------------------------------------------------------- 
#     Add shapes
#     signal_signals = []
#     for i in df.index:
#         if i in signal_days:
#             signal_signals.append(df['Adj Close'][i])
#         else:
#             signal_signals.append(np.nan)
#     signal_df = pd.DataFrame({'signal_signals':signal_signals})
    
    

    
#     x = np.arange(len(df.index)) #x軸を0からデータの個数まで1ずつ増える数列
#     labels = df.index #x軸ラベルを日付にする

    
#     trace0 = go.Candlestick(x = df.index, open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'],
#                          xaxis='x1', yaxis = 'y1', name  = stock_code)
#     trace1 = go.Scatter(x = df.index, y = df['SMA'+str(short_sma)], mode = 'lines',xaxis='x1',yaxis = 'y1')
#     trace2 = go.Scatter(x = df.index, y = df['SMA'+str(long_sma)], mode = 'lines',xaxis='x1',yaxis = 'y1')
#     trace3 = go.Scatter(x = df.index, y = df['SMA'+str(verylong_sma)], mode = 'lines',xaxis='x1',yaxis = 'y1')
#     trace4 = go.Scatter(x = df.index, y = df['Upper'], mode = 'lines',xaxis='x1',yaxis = 'y1')
#     trace5 = go.Scatter(x = df.index, y = df['Lower'], mode = 'lines',xaxis='x1',yaxis = 'y1')
#     trace6 = go.Scatter(x = df.index, y = df['Standard_deviation_normalization'], 
#                                                   mode = 'lines',xaxis='x2',yaxis = 'y2')
#     trace7 = go.Scatter(x = df.index, y = df['daviation_rate'], mode = 'lines',xaxis='x3',yaxis = 'y3')
    
#     trace8 = go.Scatter(x = df.index, y = signal_df ,mode='markers',marker_size=10000, yaxis = 'y1')
     
        
#     data = [trace0, trace1, trace2, trace3, trace6, trace7]
        
    
#     # レイアウトの指定
# #     layout = go.Layout(xaxis = dict(domain=[0, 0.45]), yaxis=dict(domain=[0, 0.45]),
# #                        xaxis2 = dict(domain=[0, 0.45]), yaxis2 = dict(domain=[0, 0.45]) ,
# #                        xaxis3 = dict(domain=[0, 0.45]), yaxis3 = dict(domain=[0, 0.45]) )
    
# #     fig = go.Figure(data=data, layout=layout)
# #     fig.show()
    
    
    
#     fig = make_subplots(rows=3, cols=1
#                               )
    
#     fig.append_trace(trace0, 1, 1)
#     fig.append_trace(trace1, 1, 1)
#     fig.append_trace(trace2, 1, 1)
#     fig.append_trace(trace3, 1, 1)
#     fig.append_trace(trace8, 1, 1)
#     fig.append_trace(trace6, 2, 1)
#     fig.append_trace(trace7, 3, 1)
    
    
    


   
        
        
    
    
   
                
                
    
    
#     make figure
    df['70'] = 70
    df['30'] = 30
    df['50'] = 50
    df['0'] = 0
    
    signal_signals = []
    for i in df.index:
        if i in signal_days:
            signal_signals.append(df['Adj Close'][i])
        else:
            signal_signals.append(np.nan)
            
    buy_signals = []
    for i in df.index:
        if i in buy_days:
            buy_signals.append(df['Adj Close'][i])
        else:
            buy_signals.append(np.nan)
            
    sell_signals = []
    for i in df.index:
        if i in sell_days:
            sell_signals.append(df['Adj Close'][i])
        else:
            sell_signals.append(np.nan)
            
           
    if buy_signals.count(np.nan) == len(buy_signals):
        continue
        
    apds = [
    mpl.make_addplot( df['slope_sma'] , panel=4, ylabel='slope_sma'),
    mpl.make_addplot(df['Upper'], type='line' ,color="r"),
    mpl.make_addplot(df['Lower'], type='line',color="r"),
    mpl.make_addplot(df['Standard_deviation_normalization'], panel=5, type='line', ylabel='normalizationSTD'),
    mpl.make_addplot( df['daviation_rate'] , panel=2, type='line', ylabel='daviation_rate'),
    mpl.make_addplot( df['range'+str(14)] , panel=3, type='line', ylabel='range'),
    mpl.make_addplot(signal_signals, type='scatter', markersize=10000, marker='|', color="y"),
    mpl.make_addplot(buy_signals, type='scatter', markersize=10000, marker='|', color="b"),
    mpl.make_addplot(sell_signals, type='scatter', markersize=10000, marker='|', color="r"),
    ]

#     if buy_signals.count(np.nan) != len(buy_signals):
#         apds.appned(mpl.make_addplot(buy_signals, type='scatter', markersize=10000, marker='|', color="b"))
    
        
#     print(all([i == np.nan for i in buy_signals]))
#     if np.isnan(all(buy_signals)) == False:
#         print('pppppp')
#     print('length of the array')
#     print(len(buy_signals))
#     print(buy_signals)
#     print(len(sell_signals))
#     print(buy_signals.count(np.nan))

#     fig = mpl.plot(df, type='candle',figsize=(30, 30), style='yahoo',mav=(short_sma, long_sma, verylong_sma)
#                        , addplot=apds, title=str(stock_code) ,panel_ratios=(2,1) ,volume = True)
    
    
    
    



columns = ['trades', 'Total return', 'Average Gain', 'Average Loss', '勝率' ,'Average gain and profit','average holding period']
df = pd.DataFrame(totalresult, columns=columns, index = sign)
df.to_csv('./volatility_result.csv')

print(df)
        

[*********************100%***********************]  1 of 1 completed
ATVI
buy in 2015-05-07 00:00:00 at 23.596202850341797
up stopline1 in 2015-05-20 00:00:00 at 23.596202850341797
sell in 2015-06-30 00:00:00 at 23.596202850341797 | profit_two is 0.0
buy in 2015-08-05 00:00:00 at 27.75284767150879
sell in 2015-08-24 00:00:00 at 25.010000228881836 | profit_one is -9.88312073446349
buy in 2017-02-10 00:00:00 at 46.026947021484375
up stopline1 in 2017-03-28 00:00:00 at 46.026947021484375
up stopline2 in 2017-04-27 00:00:00 at 48.328294372558595
up stopline3 in 2017-05-15 00:00:00 at 50.629641723632815
up stopline4 in 2017-06-22 00:00:00 at 50.971981620788576
up stopline5 in 2018-01-19 00:00:00 at 55.354791259765626
up stopline6 in 2018-01-19 00:00:00 at 55.354791259765626
up stopline6 in 2018-01-22 00:00:00 at 55.927398681640625
up stopline6 in 2018-01-23 00:00:00 at 56.12349853515625
up stopline6 in 2018-01-26 00:00:00 at 56.88435668945313
up stopline6 in 2018-01-31 00:00:00 at 58.147241

/Users/suzukiakiratarou/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:267: RuntimeWarning: invalid value encountered in double_scalars


[*********************100%***********************]  1 of 1 completed
ALXN
buy in 2017-08-28 00:00:00 at 139.02000427246094
up stopline1 in 2017-09-13 00:00:00 at 139.02000427246094
sell in 2017-09-28 00:00:00 at 139.02000427246094 | profit_two is 0.0
buy in 2019-04-03 00:00:00 at 139.99000549316406
sell in 2019-04-17 00:00:00 at 131.59060516357422 | profit_two is -5.999999999999995
buy in 2020-12-15 00:00:00 at 157.91000366210938
Date:1531 Last Day Buy at the price 155.6999969482422
[*********************100%***********************]  1 of 1 completed
ALGN
buy in 2015-06-03 00:00:00 at 63.400001525878906
sell in 2015-07-27 00:00:00 at 59.59000015258789 | profit_one is -6.009465743838871
buy in 2016-04-04 00:00:00 at 73.33999633789062
up stopline1 in 2016-05-20 00:00:00 at 73.33999633789062
up stopline2 in 2016-06-23 00:00:00 at 77.00699615478516
up stopline3 in 2016-07-29 00:00:00 at 80.67399597167969
up stopline4 in 2016-08-12 00:00:00 at 81.30250129699706
up stopline5 in 2017-03-15 00

buy in 2021-01-20 00:00:00 at 1886.9000244140625
Date:1531 Last Day Buy at the price 1901.3499755859375
[*********************100%***********************]  1 of 1 completed
AMZN
buy in 2015-07-13 00:00:00 at 455.57000732421875
up stopline1 in 2015-07-17 00:00:00 at 455.57000732421875
up stopline2 in 2015-07-24 00:00:00 at 478.3485076904297
sell in 2015-08-24 00:00:00 at 463.5799865722656 | profit_one is 1.7582323505213493
buy in 2016-07-08 00:00:00 at 745.8099975585938
up stopline1 in 2016-09-06 00:00:00 at 745.8099975585938
up stopline2 in 2016-09-28 00:00:00 at 783.1004974365235
sell in 2016-10-28 00:00:00 at 782.0 | profit_one is 4.852442654278444
buy in 2017-02-21 00:00:00 at 856.4400024414062
up stopline1 in 2017-04-04 00:00:00 at 856.4400024414062
up stopline2 in 2017-05-01 00:00:00 at 899.2620025634766
up stopline3 in 2017-07-20 00:00:00 at 942.0840026855469
sell in 2017-08-29 00:00:00 at 940.0 | profit_one is 9.756666820839044
buy in 2017-10-27 00:00:00 at 1100.949951171875
up 

buy in 2020-07-31 00:00:00 at 105.72808837890625
up stopline1 in 2020-08-06 00:00:00 at 105.72808837890625
up stopline2 in 2020-08-20 00:00:00 at 111.01449279785157
up stopline3 in 2020-08-31 00:00:00 at 116.30089721679688
sell in 2020-09-08 00:00:00 at 113.94999694824219 | profit_one is 7.776465739047911
[*********************100%***********************]  1 of 1 completed
AMAT
buy in 2019-03-21 00:00:00 at 40.250850677490234
sell in 2019-03-28 00:00:00 at 37.83579963684082 | profit_two is -6.000000000000005
buy in 2021-01-07 00:00:00 at 94.55999755859375
up stopline1 in 2021-01-12 00:00:00 at 94.55999755859375
up stopline2 in 2021-01-14 00:00:00 at 99.28799743652344
sell in 2021-01-27 00:00:00 at 99.28799743652344 | profit_two is 5.000000000000004
[*********************100%***********************]  1 of 1 completed
ASML
buy in 2019-03-15 00:00:00 at 188.0693817138672
up stopline1 in 2019-04-08 00:00:00 at 188.0693817138672
sell in 2019-05-13 00:00:00 at 188.0693817138672 | profit_two 

...

up stopline6 in 2018-06-19 00:00:00 at 323.9840087890625
up stopline6 in 2018-06-20 00:00:00 at 333.40800781250005
up stopline6 in 2018-07-09 00:00:00 at 335.17600097656253
sell in 2018-07-30 00:00:00 at 335.17600097656253 | profit_two is 59.615216062020494
buy in 2020-06-22 00:00:00 at 468.0400085449219
up stopline1 in 2020-07-06 00:00:00 at 468.0400085449219
up stopline2 in 2020-07-10 00:00:00 at 491.44200897216797
sell in 2020-07-20 00:00:00 at 489.1400146484375 | profit_one is 4.508162917335401
[*********************100%***********************]  1 of 1 completed
NVDA
buy in 2015-08-10 00:00:00 at 23.082338333129883
sell in 2015-08-20 00:00:00 at 21.69739803314209 | profit_two is -6.000000000000005
buy in 2015-11-06 00:00:00 at 30.75322914123535
up stopline1 in 2015-12-04 00:00:00 at 30.75322914123535
sell in 2016-01-07 00:00:00 at 30.739999771118164 | profit_one is -0.043017824425628515
buy in 2016-06-23 00:00:00 at 47.74141311645508
sell in 2016-06-27 00:00:00 at 44.87692832946777

[*********************100%***********************]  1 of 1 completed
PYPL
buy in 2016-09-15 00:00:00 at 40.83000183105469
up stopline1 in 2016-10-21 00:00:00 at 40.83000183105469
sell in 2016-11-04 00:00:00 at 40.29999923706055 | profit_one is -1.2980714431196194
buy in 2017-03-20 00:00:00 at 43.15999984741211
up stopline1 in 2017-04-27 00:00:00 at 43.15999984741211
up stopline2 in 2017-04-28 00:00:00 at 45.317999839782715
up stopline3 in 2017-05-31 00:00:00 at 47.47599983215333
up stopline4 in 2017-07-12 00:00:00 at 48.06749935150146
up stopline5 in 2017-09-22 00:00:00 at 52.064001464843756
up stopline6 in 2017-09-22 00:00:00 at 52.064001464843756
up stopline6 in 2017-10-05 00:00:00 at 52.655999755859376
up stopline6 in 2017-10-06 00:00:00 at 52.840002441406256
up stopline6 in 2017-10-09 00:00:00 at 52.984002685546876
up stopline6 in 2017-10-11 00:00:00 at 54.2239990234375
up stopline6 in 2017-10-12 00:00:00 at 55.08800048828125
up stopline6 in 2017-10-20 00:00:00 at 56.7760009765625


[*********************100%***********************]  1 of 1 completed
ROST
buy in 2019-04-04 00:00:00 at 96.48626708984375
sell in 2019-05-28 00:00:00 at 90.69709106445312 | profit_two is -6.000000000000005
buy in 2019-09-05 00:00:00 at 108.52423858642578
up stopline1 in 2019-11-26 00:00:00 at 108.52423858642578
up stopline2 in 2020-01-09 00:00:00 at 113.95045051574708
sell in 2020-01-29 00:00:00 at 113.95045051574708 | profit_two is 5.000000000000004
[*********************100%***********************]  1 of 1 completed
SGEN
buy in 2018-06-01 00:00:00 at 61.619998931884766
up stopline1 in 2018-06-06 00:00:00 at 61.619998931884766
up stopline2 in 2018-06-12 00:00:00 at 64.70099887847901
sell in 2018-06-18 00:00:00 at 64.33999633789062 | profit_one is 4.41414711644601
buy in 2018-08-14 00:00:00 at 74.62999725341797
up stopline1 in 2018-09-11 00:00:00 at 74.62999725341797
up stopline2 in 2018-10-03 00:00:00 at 78.36149711608887
sell in 2018-10-08 00:00:00 at 78.0199966430664 | profit_one is

buy in 2020-06-25 00:00:00 at 110.19000244140625
sell in 2020-07-14 00:00:00 at 103.56999969482422 | profit_one is -6.0078070604474565
buy in 2020-08-07 00:00:00 at 115.08999633789062
up stopline1 in 2020-10-13 00:00:00 at 115.08999633789062
sell in 2020-10-16 00:00:00 at 115.08999633789062 | profit_two is 0.0
buy in 2020-11-12 00:00:00 at 126.80999755859375
up stopline1 in 2020-12-01 00:00:00 at 126.80999755859375
sell in 2021-01-14 00:00:00 at 126.80999755859375 | profit_two is 0.0
[*********************100%***********************]  1 of 1 completed
TSLA
buy in 2015-06-19 00:00:00 at 52.50199890136719
up stopline1 in 2015-07-02 00:00:00 at 52.50199890136719
sell in 2015-07-08 00:00:00 at 51.86399841308594 | profit_one is -1.2151927576697252
[*********************100%***********************]  1 of 1 completed
TXN
buy in 2021-01-11 00:00:00 at 170.2774658203125
Date:1531 Last Day Buy at the price 172.4600067138672
[*********************100%***********************]  1 of 1 completed
TCO

[*********************100%***********************]  1 of 1 completed
VRTX
buy in 2017-03-29 00:00:00 at 108.01000213623047
up stopline1 in 2017-04-05 00:00:00 at 108.01000213623047
up stopline2 in 2017-05-01 00:00:00 at 113.410502243042
up stopline3 in 2017-06-02 00:00:00 at 118.81100234985352
up stopline4 in 2017-07-19 00:00:00 at 135.73650207519532
up stopline5 in 2017-07-21 00:00:00 at 130.53599853515627
up stopline6 in 2017-07-21 00:00:00 at 130.53599853515627
up stopline6 in 2017-07-24 00:00:00 at 132.88800048828125
up stopline6 in 2018-01-08 00:00:00 at 125.51199951171876
up stopline6 in 2018-01-10 00:00:00 at 125.90400390625001
up stopline6 in 2018-01-11 00:00:00 at 126.71999511718751
up stopline6 in 2018-01-22 00:00:00 at 129.52800292968752
up stopline6 in 2018-01-23 00:00:00 at 133.26400146484374
up stopline6 in 2018-01-24 00:00:00 at 133.95999755859376
up stopline6 in 2018-01-26 00:00:00 at 135.36800537109374
up stopline6 in 2018-02-01 00:00:00 at 138.39200439453126
up stopli